In [1]:
from scipy.constants import e, m_e, c, epsilon_0
from scipy.constants import Boltzmann as k_b
import numpy as np

### Beam specifications

In [2]:
# Lab frame quantities
slice_current = 100. # Amps (no idea if this is close)
slice_length = 4e-6 

In [3]:
gamma0 = 42.66  # assumed exact
beta0 = np.sqrt(1. - 1. / (gamma0 * gamma0))

In [4]:
# Initial Courant-Snyder parameters (beam at the waist initially):
alpha_x_ini = 0.0
beta_x_ini = 4.5  # m
gamma_x_ini = (1. + alpha_x_ini * alpha_x_ini) / beta_x_ini
eps_n_rms_x = 5.0e-6  # m-rad, normalized rms emittance
eps_rms_x = eps_n_rms_x / (gamma0 * beta0)

In [5]:
x_rms_ini = np.sqrt(eps_rms_x * beta_x_ini)
xp_rms_ini = np.sqrt(eps_rms_x * gamma_x_ini)  # lab frame value

In [6]:
v_rms_transverse = gamma0 * beta0 * c * xp_rms_ini 

In [7]:
K = 1. / (4 * np.pi * epsilon_0)

### Derived Beam Quantities

In [9]:
electrons_in_slice = slice_current * (slice_length / c) / e

In [10]:
transverse_temperature_rms = m_e * c**2 / k_b * xp_rms_ini**2
transverse_density_rms = electrons_in_slice / ((2 * np.pi)**(1.5) * x_rms_ini * x_rms_ini * (slice_length * gamma0) )

In [11]:
debye_length = np.sqrt(epsilon_0 * k_b * transverse_temperature_rms / (e**2 * transverse_density_rms))

In [12]:
plasma_frequency = np.sqrt(transverse_density_rms * e**2 / (m_e * epsilon_0))

In [21]:
def electric_field(r, Q):
    K = 1. / (4 * np.pi * epsilon_0)
    return K * Q * e / r

In [22]:
def normalized_velocity_kick(r0, dt, v_rms, Q):
    dvn = electric_field(r0, Q)
    dvn *= dt / v_rms
    dvn *= e / m_e
    
    return dvn

# Plotting

In [23]:
import matplotlib.pyplot as plt
from matplotlib import ticker, colors

In [24]:
%matplotlib widget

In [25]:
# Simulation Steps
Nsteps = 5000
L_mod = 3.7  # m, modulator section length in the lab frame
T_mod = L_mod / (gamma0 * beta0 * c)  # sim time in the _beam_ frame

In [26]:
print('debye length:', debye_length)
print('Plasma period:', 2 * np.pi / plasma_frequency)

debye length: 1.119202194575358e-05
Plasma period: 1.4532479615901866e-09


In [27]:
plt.figure()

current_dt = T_mod / np.float64(Nsteps)
current_v_rms = v_rms_transverse
current_Q = 79

plt.title(r'$ \frac{dv_{e-}}{v_{e-}^{rms}} = \frac{q_e E_{ion}(r_0)}{v_{e-}^{rms} m_{e-}} dt$', fontsize=24)

r_0_vals = np.linspace(1e-14, 1e-11, 500)
plt.plot(r_0_vals * 1e12, normalized_velocity_kick(r_0_vals, current_dt, current_v_rms, current_Q), label=r'$Q_{ion}$=79')
plt.scatter(np.array([1e-13, 5e-15]) * 1e12, normalized_velocity_kick(np.array([1e-13, 5e-15]), current_dt, current_v_rms, current_Q))
plt.plot(r_0_vals * 1e12, normalized_velocity_kick(r_0_vals, current_dt, current_v_rms, 1), label=r'$Q_{ion}$=1')
plt.yscale('log')
plt.ylabel(r'$dv_{e-}$ / $v_{e-}^{rms}$', fontsize=16)
plt.xlabel('$r_0$ (pm)', fontsize=16)
plt.legend()
plt.annotate('dt={:2.2e} s'.format(current_dt), xy=(0.6, 0.65), xycoords='figure fraction')
plt.annotate(r'$v_{e-}^{rms}$='+'{:2.2e} m/s'.format(current_v_rms), xy=(0.6, 0.6), xycoords='figure fraction')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Quick, Order-of-magnitude Check on delta-f weight update limitation impo

In [28]:
v_norm = v_rms_transverse * (1 / gamma0) / c / (gamma0 * beta0)

In [29]:
(e / m_e) * (1 * x_rms_ini + v_norm * 1) * electric_field(1e-13, 79)*e / (gamma0 * beta0 * eps_n_rms_x)

0.10975798405410368

# Look at timestep / r0 limitations for drift case assuming typical cell crossing limits

In [30]:
dx = np.sqrt(eps_rms_x * 20) / 32

In [31]:
dx / (v_rms_transverse * 6)

3.864540158077874e-12

In [32]:
current_dt

5.787746730526589e-14

In [33]:
3.864540158077874e-12 / current_dt

66.77106545963639

In [34]:
# cell size if Twiss beta goes up to 20
number_of_cells = 32
cell_size = np.sqrt(eps_rms_x * 20) / number_of_cells
# dt_min = cell_size / v_max

In [35]:
fig, ax1 = plt.subplots(1, 1)

current_dt = T_mod / np.float64(Nsteps)
current_v_rms = v_rms_transverse
current_Q = 79

dt_vals = np.linspace(10e-14, 3e-12, 500)

dt_ratio = dx / (current_v_rms * 6 + current_v_rms *normalized_velocity_kick(1e-13, dt_vals, current_v_rms, current_Q)) / dt_vals
ax1.plot(1e12*dt_vals, 1/dt_ratio, label=r'$r_0$ = 100 fm')

dt_ratio = dx / (current_v_rms * 6 + current_v_rms*normalized_velocity_kick(5e-15, dt_vals, current_v_rms, current_Q)) / dt_vals
ax1.plot(1e12*dt_vals, 1/dt_ratio, label=r'$r_0$ = 5 fm')

ax1.set_xlabel('dt (ps)')
ax1.set_ylabel(r'$\frac{dx}{v_{max}(r_0)}$ / dt')
ax1.legend()
                 
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
fig, ax1 = plt.subplots(1, 1, figsize=(12, 6))

ax1.set_title(r'dv * dt  / $\lambda_D$')
current_dt = T_mod / np.float64(Nsteps)
current_v_rms = v_rms_transverse
current_Q = 79

r_0_vals = np.linspace(1e-14, 1e-11, 500)
dt_vals = np.linspace(3e-14, 1e-12, 500)
X, Y = np.meshgrid(r_0_vals, dt_vals)
Z = normalized_velocity_kick(X, Y, current_v_rms, current_Q) * current_v_rms * Y / debye_length
r0_contour = ax1.contourf(X * 1e12, Y * 1e12, Z, levels=64, locator=ticker.LogLocator(subs='all'))
ax1.set_xlabel(r'$r_0$ (pm)')
ax1.set_ylabel('dt (ps)')
fig.colorbar(r0_contour)

                 
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

current_dt = T_mod / np.float64(Nsteps)
current_v_rms = v_rms_transverse
current_Q = 79

r_0_vals = np.linspace(1e-15, 1e-13, 500)
dt_vals = np.linspace(1e-15, 1e-12, 500)
X, Y = np.meshgrid(r_0_vals, dt_vals)

r0_contour = ax1.contourf(X, Y, normalized_velocity_kick(X, Y, current_v_rms, current_Q), levels=64, locator=ticker.LogLocator(subs='all'))
ax1.set_xlabel(r'$r_0$')
ax1.set_ylabel('time step')
fig.colorbar(r0_contour)
dt_ratio = dx / (current_v_rms * 6 + current_v_rms*normalized_velocity_kick(X, Y, current_v_rms, current_Q)) / Y
Z = 1/dt_ratio
lev_exp = np.linspace(np.floor(np.log10(Z.min())-1),
                   np.ceil(np.log10(Z.max())+1), 10)
level = np.power(10, lev_exp)
dt_contour = ax2.contourf(X, Y, 1/dt_ratio, level, norm=colors.LogNorm())
# dt_contour = ax2.contourf(X, Y, 1/dt_ratio, levels=64, locator=ticker.LogLocator(subs='all'))
fig.colorbar(dt_contour)
ax2.set_xlabel(r'$r_0$')
ax2.set_ylabel('time step')
                 
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
lev_exp

array([-5.        , -4.33333333, -3.66666667, -3.        , -2.33333333,
       -1.66666667, -1.        , -0.33333333,  0.33333333,  1.        ])

In [39]:
np.linspace(np.floor(np.log10(Z.min())-1),
                   np.ceil(np.log10(Z.max())+1), 10)

array([-5.        , -4.33333333, -3.66666667, -3.        , -2.33333333,
       -1.66666667, -1.        , -0.33333333,  0.33333333,  1.        ])